### Chargement des données
Dans cette section, nous allons charger les données à partir d'un fichier Excel. Nous utiliserons la bibliothèque `ExcelReaders` pour lire le fichier et la bibliothèque `DataFrames` pour manipuler les données.


In [ ]:
using DataFrames
using ExcelReaders
using Plots
using DataValues
using XLSX
using Statistics
using Tables
import Plots: plot!, vline!, plot


In [ ]:
mutable struct GV
    numero::Float64
    palier::String
    unite::String
    sous_unite::String
    circuit::String
    ref::String
    reg_ref::Vector{String}
    avant_RGV::Bool
    numero_suc_ou_pred::Union{Float64, Missing}
    maintenances::DataFrame
    IND_COL_1::DataFrame
    PE_max_IND_COL_1::String
    IND_COL_2::DataFrame
    IND_COL_3::DataFrame
    IND_ENC::DataFrame
    date_max::Float64
    date_RGV::Union{Float64, Missing}
    ylim_IND_COL::Union{Float64, Int}
    pb::Bool

    function GV(numero::Float64, palier::String, unite::String, sous_unite::String, circuit::String, 
                ref::String, reg_ref::Vector{String}, avant_RGV::Bool, 
                numero_suc_ou_pred::Union{Float64, Missing}, maintenances::DataFrame, 
                IND_COL_1::DataFrame, PE_max_IND_COL_1::String, IND_COL_2::DataFrame, 
                IND_COL_3::DataFrame, IND_ENC::DataFrame, date_max::Float64, date_RGV::Union{Float64, Missing}, 
                ylim_IND_COL::Union{Float64, Int})
        new(numero, palier, unite, sous_unite, circuit, ref, reg_ref, avant_RGV, numero_suc_ou_pred, 
        maintenances, IND_COL_1, PE_max_IND_COL_1, IND_COL_2, IND_COL_3, IND_ENC, date_max, date_RGV,
        ylim_IND_COL)
    end
end


In [ ]:
mutable struct NAryTreeNode
    parent::Union{NAryTreeNode, Missing}
    value::Union{String, Float64, Missing}
    children::Vector{NAryTreeNode}

    function NAryTreeNode(parent::Union{NAryTreeNode, Missing}, value::Union{String, Float64, Missing}, children::Vector{NAryTreeNode})
        new(parent, value, children)
    end
end

### Avant de commencer à éxécuter les cellules du code
- Copier dans la variable `chemin_données`, le chemin d'accès vers le dossier `Données_EDF_240611`
- Copier dans la variable `chemin_projet`, le chemin d'accès vers le dossier `JuliaStatsProject`

In [ ]:
chemin_données = "/home/AD/faidy/JuliaStatsProject/data/real_data/"
chemin_projet = "/home/AD/faidy/JuliaStatsProject/"

Dans cette section, nous chargeons les données à partir des fichier Excel `IND-COL-1.xls` et `IND-COL-2.xls` situé dans le répertoire spécifié.

In [ ]:
# Transformer le contenu de la feuille de IND-COL-1.xls en une matrice
data_matrix_ind_col1 = readxlsheet(string(chemin_données, "Données_EDF_240611/Colmatage/IND-COL-1.xls"), "IND-COL-1")

In [ ]:
# Transformer le contenu de la feuille de IND-COL-2.xls en une matrice
data_matrix_ind_col2 = readxlsheet(string(chemin_données, "Données_EDF_240611/Colmatage/IND-COL-2.xls"), "IND-COL-2")

In [ ]:
# Les cases vides de la colonne VALEUR des fichiers IND-COL-1.xls et IND-COL-2.xls ont été remplis par un objet DataValue{Union{}} 
# qu'on le remplace par l'objet missing qui représente une donnée manquante en Julia
isNA(x) = typeof(x) == DataValue{Union{}}
valeurs = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col1)

data_matrix_ind_col1 = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col1)
data_matrix_ind_col2 = replace( x -> isNA(x) ? missing : x, data_matrix_ind_col2)

### Conversion de Matrice en DataFrame

La fonction `MatrixToDataFrame(mat)` prend en entrée une matrice `mat` et retourne un DataFrame `DF_mat` en excluant la première ligne de `mat` comme les noms de colonnes.

#### Paramètres :

- `mat` : Une matrice représentant les données à convertir en DataFrame.

#### Sortie :

Un objet DataFrame `DF_mat` où les données de la première ligne de `mat` sont utilisées comme noms de colonnes.

Cette fonction est utile pour convertir des données structurées sous forme de matrice en un format plus facile à manipuler et à analyser à l'aide de la bibliothèque `DataFrames` en Julia.


In [ ]:
function MatrixToDataFrame(mat)
    DF_mat = DataFrame(
        mat[2:end, 1:end],
        string.(mat[1, 1:end])
    )
    return DF_mat
end

In [ ]:
df1 = MatrixToDataFrame(data_matrix_ind_col1)

In [ ]:
df2 = MatrixToDataFrame(data_matrix_ind_col2)

### Suppression des valeurs manquantes

Les lignes suivantes suppriment les lignes de `df1` et `df2` où la colonne `:VALEUR` contient des valeurs manquantes.


In [ ]:
df1 = dropmissing(df1, :VALEUR)

In [ ]:
df2 = dropmissing(df2, :VALEUR)

In [ ]:
nettoyages = readxlsheet(string(chemin_données, "Données_EDF_240611/Nettoyages/NETTOYAGES.xls"), "NETTOYAGES")
df_nettoyages = MatrixToDataFrame(nettoyages)

In [ ]:
df_Infos_gen = readxlsheet(string(chemin_données, "Données_EDF_240611/Informations générales 9-7/HEURES.xls"), "HEURES")
df_Infos_gen = MatrixToDataFrame(df_Infos_gen)

In [ ]:
function IND_COL_3_and_EncMatToDataFrame(mat, circuit::String)
    # Extraire les noms des colonnes
    column_names = [mat[1,1],"$(mat[2,3])_ENC", "$(mat[2,4])_ENC", "$(mat[2,5])_ENC", "$(mat[2,6])_ENC", "$(mat[2,15])_IND_COL_3", "$(mat[2,16])_IND_COL_3", "$(mat[2,17])_IND_COL_3", "$(mat[2,18])_IND_COL_3"]

    # Créer un DataFrame à partir des données extraites
    df_enc = DataFrame()
    df_ind_col_3 = DataFrame()

    # Ajouter les données au DataFrame
    df_enc[!, column_names[1]] = mat[3:end,1]

    if circuit == "C1"
        df_enc[!, column_names[2]] = mat[3:end,3]
    elseif circuit == "C2"
        df_enc[!, column_names[3]] = mat[3:end,4]
    elseif circuit == "C3"
        df_enc[!, column_names[4]] = mat[3:end,5]
    else
        df_enc[!, column_names[5]] = mat[3:end,6]
    end
    
    df_ind_col_3[!, column_names[1]] = mat[3:end,1]
    if circuit == "C1"
        df_ind_col_3[!, column_names[6]] = mat[3:end,15]
    elseif circuit == "C2"
        df_ind_col_3[!, column_names[7]] = mat[3:end,16]
    elseif circuit == "C3"
        df_ind_col_3[!, column_names[8]] = mat[3:end,17]
    else
        df_ind_col_3[!, column_names[9]] = mat[3:end,18]
    end
    return df_enc, df_ind_col_3
end


In [ ]:
# Création des enfants
U2 = NAryTreeNode(missing, "U2", Vector{NAryTreeNode}())
U3 = NAryTreeNode(missing, "U3", Vector{NAryTreeNode}())
U5 = NAryTreeNode(missing, "U5", Vector{NAryTreeNode}())
U9 = NAryTreeNode(missing, "U9", Vector{NAryTreeNode}())
U10 = NAryTreeNode(missing, "U10", Vector{NAryTreeNode}())
U18 = NAryTreeNode(missing, "U18", Vector{NAryTreeNode}())
U19 = NAryTreeNode(missing, "U19", Vector{NAryTreeNode}())
U8 = NAryTreeNode(missing, "U8", Vector{NAryTreeNode}())
U13 = NAryTreeNode(missing, "U13", Vector{NAryTreeNode}())

# Création du parent avec les enfants
P1 = NAryTreeNode(missing, "P1", Vector([U2, U3, U5, U9, U10, U18, U19, U8, U13]))

# Mettre à jour les parents des enfants
for child in P1.children
    child.parent = P1
end


In [ ]:
# Création des enfants
U1 = NAryTreeNode(missing, "U1", Vector{NAryTreeNode}())
U4 = NAryTreeNode(missing, "U4", Vector{NAryTreeNode}())
U11 = NAryTreeNode(missing, "U11", Vector{NAryTreeNode}())
U12 = NAryTreeNode(missing, "U12", Vector{NAryTreeNode}())
U14 = NAryTreeNode(missing, "U14", Vector{NAryTreeNode}())
U15 = NAryTreeNode(missing, "U15", Vector{NAryTreeNode}())
U16 = NAryTreeNode(missing, "U16", Vector{NAryTreeNode}())
U17 = NAryTreeNode(missing, "U17", Vector{NAryTreeNode}())

# Création du parent avec les enfants
P2 = NAryTreeNode(missing, "P2", Vector([U1, U4, U11, U12, U14, U15, U16, U17]))

# Mettre à jour les parents des enfants
for child in P2.children
    child.parent = P2
end

In [ ]:
# Création des enfants
U6 = NAryTreeNode(missing, "U6", Vector{NAryTreeNode}())
U7 = NAryTreeNode(missing, "U7", Vector{NAryTreeNode}())

# Création du parent avec les enfants
P3 = NAryTreeNode(missing, "P3", Vector([U6, U7]))

# Mettre à jour les parents des enfants
for child in P3.children
    child.parent = P3
end

In [ ]:
paliers = [P1, P2, P3]

for palier in paliers
    unites = palier.children
    for unite in unites
        sous_unites = Vector(unique(df_Infos_gen[df_Infos_gen.UNITE .== unite.value, :].SOUS_UNITE))
        for sous_unite in sous_unites
            push!(unite.children, NAryTreeNode(unite, sous_unite, Vector{NAryTreeNode}()))
        end
    end
end

In [ ]:

# Fonction récursive pour afficher l'arbre à partir des paliers dans un fichier texte
function print_tree_from_paliers(paliers::Vector{NAryTreeNode}, filename::String)
    open(filename, "w") do file
        for palier in paliers
            write(file, "Palier: ", string(palier.value), "\n")
            for unite in palier.children
                write(file, "   |- Unité: ", string(unite.value), "\n")
                for sous_unite in unite.children
                    write(file, "       |- Sous-unité: ", string(sous_unite.value), "\n")
                    for etat in sous_unite.children
                        write(file, "           |- ", string(etat.value), "\n")
                        for circuit in etat.children
                            write(file, "               |- ", string(circuit.value), "\n")
                            for num in circuit.children
                                if !ismissing(num)
                                    write(file, "                 |- ", string(num.value), "\n")
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end



In [ ]:
for palier in paliers
    unites = palier.children
    for unite in unites
        sous_unites = unite.children
        for sous_unite in sous_unites
            push!(sous_unite.children, NAryTreeNode(sous_unite, "avant_RGV", Vector{NAryTreeNode}()))
            push!(sous_unite.children, NAryTreeNode(sous_unite, "apres_RGV", Vector{NAryTreeNode}()))
        end
    end
end

In [ ]:
for palier in paliers
    unites = palier.children
    for unite in unites
        sous_unites = unite.children
        for sous_unite in sous_unites
            data_sous_unite_avant = filter(row -> row.UNITE .== unite.value 
                                    && row.SOUS_UNITE .== sous_unite.value
                                    && row[10] .== "ORIGINE",
                                 df_Infos_gen)
            data_sous_unite_apres = filter(row -> row.UNITE .== unite.value 
                                    && row.SOUS_UNITE .== sous_unite.value
                                    && row[10] .== "Remplacement",
                                    df_Infos_gen)
            circuits_avant = unique(data_sous_unite_avant.CIRCUIT)
            circuits_apres = unique(data_sous_unite_apres.CIRCUIT)
            for etat in sous_unite.children
                if etat.value == "avant_RGV"
                    for circuit in circuits_avant
                        push!(etat.children, NAryTreeNode(etat, circuit, Vector{NAryTreeNode}()))
                    end
                else
                    for circuit in circuits_apres
                        push!(etat.children, NAryTreeNode(etat, circuit, Vector{NAryTreeNode}()))
                    end
                end
            end
        end
    end
end

In [ ]:
function max_string(v::Vector{Any})
    # Extract numbers from the strings and find the maximum
    max_str = ""
    max_num = -Inf
    for s in v
        # Use a regular expression to extract the number
        m = match(r"E(\d+)", s)
        if m !== nothing
            num = parse(Int, m.captures[1])
            if num > max_num
                max_num = num
                max_str = s
            end
        end
    end
    return max_str
end


In [ ]:

# Spécifiez le chemin complet de votre répertoire
directory = string(chemin_données, "Données_EDF_240611/Encrassement/")

# Utilisez readdir() pour obtenir les noms des fichiers et des sous-répertoires
files = readdir(directory)

# Filtrer les fichiers pour ne garder que ceux qui ont l'extension .xlsx
xlsx_files = filter(file -> endswith(file, ".xlsx"), files)
xlsx_files = [file for file in xlsx_files if file != "~\$PERFOS_U2S4_3,48-3,47-3,49.xlsx" && file != "~\$PERFOS_U19S2_2,45-2,43-2,44.xlsx"]

In [ ]:
using Printf

function find_in_vector(substring::String, vec::Vector{String})
    # Chercher l'occurrence dans le vecteur de strings
    indices = findall(x -> occursin(substring, x), vec)    
    
    return vec[indices]
end

# Fonction pour convertir un float en string avec une virgule, formaté avec une précision de 2 décimales
function float_to_comma_string(num::Float64)::String
    # Formattage avec 2 décimales et remplacement du point par une virgule
    return replace(@sprintf("%.2f", num), "." => ",")
end


# Trouver les indices où la chaîne apparaît dans le vecteur
substring = float_to_comma_string(1.9)
indices = find_in_vector(substring, xlsx_files)
println("files found: ", indices)


In [ ]:
# Exemple de DataFrames avec heures exprimées en entiers
A = DataFrame(HEURES_MAT = [8, 12, 16])
B = DataFrame(HEURES_MAT = [])

# Fonction pour calculer le nombre de nettoyages précédents pour chaque ligne de ind_col_1
function count_previous_cleanings(ind_col_1::DataFrame, nettoyages::DataFrame)::Vector{Int}
    if isempty(nettoyages)
        return zeros(Int8, nrow(ind_col_1))
    end
    counts = Vector{Int}(undef, nrow(ind_col_1))
    if "APRES_NET" in names(ind_col_1)
        for i in 1:nrow(ind_col_1)
            counts[i] = sum(nettoyages.HEURES_MAT .< ind_col_1.HEURES_MAT[i]) + (ind_col_1.APRES_NET[i] ? 1 : 0)
        end
    else
        for i in 1:nrow(ind_col_1)
            counts[i] = sum(nettoyages.HEURES_MAT .< ind_col_1.HEURES_MAT[i]) 
        end
    end

    return counts
end

# Ajouter la nouvelle colonne au DataFrame ind_col_1
A.NETTOYAGES_PRECEDENTS = count_previous_cleanings(A, B)

# Afficher le DataFrame mis à jour
println(A)


In [ ]:
"APRES_NET" in names(df1)

In [ ]:
GVs_dict = Dict{Float64, GV}()

In [ ]:
df1

In [ ]:
df1[df1.NUMERO .== 0.54, :]

In [ ]:
for palier in paliers
    unites = palier.children
    for unite in unites
        sous_unites = unite.children
        for sous_unite in sous_unites
            data_sous_unite = filter(row -> row.UNITE .== unite.value && row.SOUS_UNITE .== sous_unite.value,
                                 df_Infos_gen)
            dict = Dict{String, DataFrame}()
            dict["avant_RGV"] = data_sous_unite[data_sous_unite[:, 10] .== "ORIGINE", :]
            dict["apres_RGV"] = data_sous_unite[data_sous_unite[:, 10] .== "Remplacement", :]
            date_RGV = missing
            if !isempty(dict["apres_RGV"])
                date_RGV = maximum(dict["avant_RGV"].HEURES_MAT)
            end
            for etat in sous_unite.children
                ss_unite_data = filter(row -> row.UNITE .== unite.value 
                                        && row.SOUS_UNITE .== sous_unite.value
                                        && row.NUMERO in dict[etat.value].NUMERO, df1)
                PE_max = max_string(ss_unite_data.PE)
                ylim_IND_COL = !isempty(ss_unite_data[ss_unite_data.PE .== PE_max, :]) ? maximum(ss_unite_data[ss_unite_data.PE .== PE_max, :].VALEUR) : 0
                for circuit in etat.children
                    num = unique(dict[etat.value][dict[etat.value].CIRCUIT .== circuit.value, :].NUMERO)
                    if length(num) == 1
                        num = num[1]
                        
                        ref = unique(dict[etat.value][dict[etat.value].CIRCUIT .== circuit.value, :].REFERENCE)[1]
                        
                        reg_ref = Vector([""])
                        
                        numero_suc_ou_pred = missing
                        if etat.value=="avant_RGV" && !isempty(dict["apres_RGV"])
                            numero_suc_ou_pred = unique(dict["apres_RGV"][dict["apres_RGV"].CIRCUIT .== circuit.value, :].NUMERO)[1]
                        end
                        if etat.value=="apres_RGV" && !isempty(dict["avant_RGV"])
                            numero_suc_ou_pred = unique(dict["avant_RGV"][dict["avant_RGV"].CIRCUIT .== circuit.value, :].NUMERO)[1]
                        end
                        
                        nettoyages = DataFrame()
                        ind_col_1 = DataFrame()
                        ind_col_2 = DataFrame()
                        df_enc, df_ind_col_3 = DataFrame(), DataFrame()
                        date_max = 0
                        if !ismissing(num)
                            nettoyages = df_nettoyages[df_nettoyages.NUMERO .== num, :]
                            nettoyages = nettoyages[:, [6, 7]] # 7 pour curatif
                            
                            ind_col_1 = df1[df1.NUMERO .== num, :]
                            if !isempty(ind_col_1)
                                PE_max = max_string(ind_col_1.PE)
                            end
                            ind_col_1 = ind_col_1[ind_col_1.PE .== PE_max, :]

                            select!(ind_col_1, Not([:ACIERISTE, :TUBISTE, :CONSTITUTION]))
                            ind_col_1_C = ind_col_1[ind_col_1.BR .== "C", :]
                            ind_col_1_F = ind_col_1[ind_col_1.BR .== "F", :]

                            if !isempty(ind_col_1_C) || !isempty(ind_col_1_F)
                                date_max = maximum(ind_col_1.HEURES_MAT)

                                # Supprimer la colonne 'B'
                                select!(ind_col_1_C, Not(:BR))
                                select!(ind_col_1_F, Not(:BR))

                                rename!(ind_col_1_C, :VALEUR => :VALEUR_CHAUD)
                                rename!(ind_col_1_F, :VALEUR => :VALEUR_FROID)

                                ind_col_1_C.nb_nettoyages_precedents = count_previous_cleanings(ind_col_1_C, nettoyages)
                                ind_col_1_F.nb_nettoyages_precedents = count_previous_cleanings(ind_col_1_F, nettoyages)

                                
                                ind_col_1 = innerjoin(ind_col_1_C, ind_col_1_F, 
                                on = [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT, :PE, :REFERENCE,
                                :nb_nettoyages_precedents], 
                                makeunique = true)

                                # Ajouter une colonne pour la moyenne
                                ind_col_1[!, :VALEUR_MOYENNE] = [mean(skipmissing([row[:VALEUR_CHAUD], row[:VALEUR_FROID]])) for row in eachrow(ind_col_1)]
                            end
                            
                            ind_col_2 = df2[df2.NUMERO .== num, :]
                            if PE_max == ""
                                ss_unite_data = filter(row -> row.UNITE .== unite.value 
                                            && row.SOUS_UNITE .== sous_unite.value
                                            && row.NUMERO in dict[etat.value].NUMERO, df2)
                                PE_max = max_string(ss_unite_data.PE)
                            end
                            ind_col_2 = ind_col_2[ind_col_2.PE .== PE_max, :]
                            select!(ind_col_2, Not([:ACIERISTE, :TUBISTE, :CONSTITUTION]))
                            ind_col_2_C = ind_col_2[ind_col_2.BR .== "C", :]
                            ind_col_2_F = ind_col_2[ind_col_2.BR .== "F", :]
                            
                            if !isempty(ind_col_2)
                                date_max = max(date_max, !isempty(ind_col_2.HEURES_MAT) ? maximum(ind_col_2.HEURES_MAT) : 0)
                                
                                # Supprimer la colonne 'B'
                                select!(ind_col_2_C, Not(:BR))
                                select!(ind_col_2_F, Not(:BR))
                                
                                rename!(ind_col_2_C, :VALEUR => :VALEUR_CHAUD)
                                rename!(ind_col_2_F, :VALEUR => :VALEUR_FROID)
                                
                                ind_col_2_C.nb_nettoyages_precedents = count_previous_cleanings(ind_col_2_C, nettoyages)
                                ind_col_2_F.nb_nettoyages_precedents = count_previous_cleanings(ind_col_2_F, nettoyages)
                                
                                
                                ind_col_2 = innerjoin(ind_col_2_C, ind_col_2_F, 
                                on = [:UNITE, :SOUS_UNITE, :CIRCUIT, :NUMERO, :HEURES_MAT, :PE, :REFERENCE,
                                :nb_nettoyages_precedents], 
                                makeunique = true)
                                # Ajouter une colonne pour la moyenne
                                ind_col_2[!, :VALEUR_MOYENNE] = [mean(skipmissing([row[:VALEUR_CHAUD], row[:VALEUR_FROID]])) for row in eachrow(ind_col_2)]
                                ylim_IND_COL = max(ylim_IND_COL, 
                                        !isempty(ind_col_2.VALEUR_CHAUD) ? maximum(ind_col_2.VALEUR_CHAUD) : 0, 
                                        !isempty(ind_col_2.VALEUR_FROID) ? maximum(ind_col_2.VALEUR_FROID) : 0)
                            end
                            
                            enc_files = find_in_vector(float_to_comma_string(num), xlsx_files)
                            if !isempty(enc_files)
                                enc_file = enc_files[1]
                                df_enc, df_ind_col_3 = IND_COL_3_and_EncMatToDataFrame(XLSX.readdata(string(directory, "$enc_file"), 
                                "Feuil1", "A1:V1000"), circuit.value)
                                
                                df_enc, df_ind_col_3 = dropmissing(df_enc, :HEURES_MAT), dropmissing(df_ind_col_3, :HEURES_MAT)

                                df_enc.nb_nettoyages_precedents = count_previous_cleanings(df_enc, nettoyages)
                                df_ind_col_3.nb_nettoyages_precedents = count_previous_cleanings(df_ind_col_3, nettoyages)

                                date_max = max(date_max, maximum(df_enc.HEURES_MAT))
                            end
                            
                        end
                        
                        # date_max = Int(floor(date_max))
                        date_max = max(date_max, maximum(filter(row -> row.UNITE .== unite.value && row.SOUS_UNITE .== sous_unite.value, df_Infos_gen).HEURES_MAT))
                        # date_max = Int(floor(date_max))

                        gv = GV(num, palier.value, unite.value, sous_unite.value, circuit.value, ref, reg_ref, 
                        etat.value=="avant_RGV", numero_suc_ou_pred, nettoyages, ind_col_1, PE_max, 
                        ind_col_2, df_ind_col_3, df_enc, date_max, date_RGV, ylim_IND_COL)
                        GVs_dict[num] = gv
                        push!(circuit.children, NAryTreeNode(circuit, num, Vector{NAryTreeNode}()))
                    else
                        num = missing
                        push!(circuit.children, NAryTreeNode(circuit, num, Vector{NAryTreeNode}()))
                    end
                    
                end
            end
        end
    end
end


In [ ]:
# Afficher l'arbre à partir des paliers
print_tree_from_paliers(paliers, "arbre.txt")

In [ ]:
circuit_shapes = Dict{String, Symbol}()
# coder le numéro de circuit en forme géométrique
circuit_shapes["C1"]= :circle
circuit_shapes["C2"]= :diamond
circuit_shapes["C3"]= :rect
circuit_shapes["C4"]= :star5

circuit_enc_colors = Dict{String, Symbol}()

circuit_enc_colors["C1"]= :pink
circuit_enc_colors["C2"]= :purple
circuit_enc_colors["C3"]= :grey
circuit_enc_colors["C4"]= :magenta


In [ ]:
function plot_with_condition(p, x, y, z, color)
    # if length(x) == 0 || length(y) == 0 || length(z) == 0
    #     return
    # else
        for i in 1:length(x) - 1
            if z[i] == z[i + 1] 
                plot!(p, x[i:i + 1], y[i:i + 1], linestyle=:dash,
                color=color, legend = false)
            end
        end
    # end
end

In [ ]:
function plot!(gv::GV, p_1_2::Plots.Plot{Plots.GRBackend}, p_3::Plots.Plot{Plots.GRBackend}, p_enc::Plots.Plot{Plots.GRBackend})
    if !isempty(gv.IND_COL_1)
        # plot!(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_CHAUD, linestyle=:dash,
        # color=:yellow, legend = false)
        plot_with_condition(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_CHAUD, 
        gv.IND_COL_1.nb_nettoyages_precedents, :yellow)
        scatter!(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_CHAUD, alpha=.75, 
        label = false, markershape=circuit_shapes[gv.circuit], markercolor=:yellow, 
        legend = false, markerstrokewidth = 0.5, markersize=6)
        
        # plot!(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_FROID, linestyle=:dash,
        # color=:green, legend = false)
        plot_with_condition(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_FROID, 
        gv.IND_COL_1.nb_nettoyages_precedents, :green)
        scatter!(p_1_2, gv.IND_COL_1.HEURES_MAT, gv.IND_COL_1.VALEUR_FROID, alpha=.75, 
        label = false, markershape=circuit_shapes[gv.circuit], markercolor=:green, 
        legend = false, markerstrokewidth = 0.5)
    end
    
    if !isempty(gv.IND_COL_2)
        if gv.numero == 1.94
            savefig(p_1_2, "1,94.png")
        end
        # plot!(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_CHAUD, linestyle=:dash,
        # color=:red, legend = false)
        plot_with_condition(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_CHAUD, 
        gv.IND_COL_2.nb_nettoyages_precedents, :red)
        scatter!(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_CHAUD, 
        alpha=.75, label = false, markershape=circuit_shapes[gv.circuit], markercolor=:red, 
        legend = false, markerstrokewidth = 0.5)

        # plot!(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_FROID, linestyle=:dash,
        # color=:blue)
        plot_with_condition(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_FROID, 
        gv.IND_COL_2.nb_nettoyages_precedents, :blue)
        scatter!(p_1_2, gv.IND_COL_2.HEURES_MAT, gv.IND_COL_2.VALEUR_FROID, alpha=.75, 
        label = false, markershape=circuit_shapes[gv.circuit], markercolor=:blue, 
        legend = false, markerstrokewidth = 0.5)
    end

    if !isempty(gv.IND_COL_3)
        # plot!(p_3, gv.IND_COL_3.HEURES_MAT, gv.IND_COL_3[:, 2],
        # color= circuit_enc_colors[gv.circuit], label = false, linestyle=:dash)
        plot_with_condition(p_3, gv.IND_COL_3.HEURES_MAT, gv.IND_COL_3[:, 2], 
        gv.IND_COL_3.nb_nettoyages_precedents, circuit_enc_colors[gv.circuit])
        scatter!(p_3, gv.IND_COL_3.HEURES_MAT, gv.IND_COL_3[:, 2], 
        alpha=.75, label = gv.circuit, markershape=circuit_shapes[gv.circuit],
        markercolor=circuit_enc_colors[gv.circuit], markerstrokewidth = 0.5)
    end

    if !isempty(gv.IND_ENC)
        # plot!(p_enc, gv.IND_ENC.HEURES_MAT, gv.IND_ENC[:, 2],
        # color= circuit_enc_colors[gv.circuit], label = false, linestyle=:dash)
        plot_with_condition(p_enc, gv.IND_ENC.HEURES_MAT, gv.IND_ENC[:, 2], 
        gv.IND_ENC.nb_nettoyages_precedents, circuit_enc_colors[gv.circuit])
        scatter!(p_enc, gv.IND_ENC.HEURES_MAT, gv.IND_ENC[:, 2], 
        alpha=.75, label = gv.circuit, markershape=circuit_shapes[gv.circuit], 
        markercolor=circuit_enc_colors[gv.circuit], markerstrokewidth = 0.5)
    end
    
    if !isempty(gv.maintenances)
        for i in 1:length(gv.maintenances.HEURES_MAT)
            linestyle = gv.maintenances[i, 2] == 1 ? :dot : :dash
            vline!(p_1_2, [gv.maintenances[i, 1]], color=:black, linestyle=linestyle, label=false)
            vline!(p_3, [gv.maintenances[i, 1]], color=:black, linestyle=linestyle, label=false)
            vline!(p_enc, [gv.maintenances[i, 1]], color=:black, linestyle=linestyle, label=false)
        end
    end
end

In [ ]:
function plot(sous_unite::NAryTreeNode)
    p_1_2_avant = plot(
        ylabel="IND-COL",  
        titlefont=10
        )
    p_3_avant = plot(
        ylabel="IND-COL-3",  
        titlefont=10
        )
    p_enc_avant = plot(
        xlabel="HEURES_MAT", 
        ylabel="IND-ENC",  
        titlefont=10
        )
    p_1_2_apres = plot(
        ylabel="IND-COL",  
        titlefont=10
        )
    p_3_apres = plot(
        ylabel="IND-COL-3",  
        titlefont=10
        )
    p_enc_apres = plot(
        xlabel="HEURES_MAT", 
        ylabel="IND-ENC",  
        titlefont=10,
        )
    titre_avant = ""
    titre_apres = ""
    date_RGV = missing
    ylim_IND_COL = 0
    date_max = 0
    for etat in sous_unite.children
        for circuit in etat.children
            if !ismissing(first(circuit.children))
                numero = first(circuit.children).value
                gv = GVs_dict[numero]
                circuit = gv.circuit
                PE = gv.PE_max_IND_COL_1

                if etat.value=="avant_RGV" 
                    plot!(gv, p_1_2_avant, p_3_avant, p_enc_avant)
                    titre_avant = string(titre_avant, "$circuit-$numero-$PE ") 
                    date_RGV = gv.date_RGV
                else
                    plot!(gv, p_1_2_apres, p_3_apres, p_enc_apres)
                    titre_apres = string(titre_apres, "$circuit-$numero-$PE ")
                end

                ylim_IND_COL = max(ylim_IND_COL, gv.ylim_IND_COL)
                date_max = max(date_max, gv.date_max)
            end
        end
    end

    if !ismissing(date_RGV)
        vline!(p_1_2_avant, [date_RGV], color=:black, linestyle=:solid, label="RGV")
        vline!(p_3_avant, [date_RGV], color=:black, linestyle=:solid, label="RGV")
        vline!(p_enc_avant, [date_RGV], color=:black, linestyle=:solid, label="RGV")
    else
        vline!(p_1_2_avant, [0], color=:black, label = false)
        vline!(p_3_avant, [0], color=:black, label = false)
        vline!(p_enc_avant, [0], color=:black, label = false)
    end

    ylims!(p_1_2_avant, 0, ylim_IND_COL*11/10)
    ylims!(p_1_2_apres, 0, ylim_IND_COL*11/10)

    xlims!(p_1_2_avant, 0, date_max*11/10)
    xlims!(p_3_avant, 0, date_max*11/10)
    xlims!(p_enc_avant, 0, date_max*11/10)

    xlims!(p_1_2_apres, 0, date_max*11/10)
    xlims!(p_3_apres, 0, date_max*11/10)
    xlims!(p_enc_apres, 0, date_max*11/10)

    unite = sous_unite.parent.value
    sous_unite = sous_unite.value
    titre_avant = string("$unite-$sous_unite ", titre_avant)
    titre_apres = string("$unite-$sous_unite ", titre_apres)

    title!(p_1_2_avant, titre_avant)
    title!(p_1_2_apres, titre_apres)
    title!(p_3_avant, titre_avant)
    title!(p_enc_avant, titre_avant)
    title!(p_3_apres, titre_apres)
    title!(p_enc_apres, titre_apres)

    p = plot(p_1_2_avant, p_1_2_apres, p_3_avant, p_3_apres, p_enc_avant, p_enc_apres, layout=(3, 2), size=(1800, 1200))

    # Chemin du répertoire où vous souhaitez enregistrer le fichier (relatif au répertoire de travail actuel)
    directory = string(chemin_projet, "plots/Visualisation")
            
    # Combiner le chemin de répertoire et le nom du fichier
    filename = joinpath(directory, string("$unite", " ", "$sous_unite.png"))
    # savefig(p, filename)
end

In [ ]:
for palier in paliers
    for unite in palier.children
        for sous_unite in unite.children
            plot(sous_unite)
        end
    end
end

In [ ]:
GVs_dict[1.92].IND_COL_2

In [ ]:
BigDF = DataFrame(
    PALIER = String[],
    UNITE = String[],
    SOUS_UNITE = String[],
    CIRCUIT = String[],
    NUMERO = Float64[],
    REF = String[],
    REG_REF = String[],
    AVANT_RGV = Bool[],
    HEURES_MAT = Int[],
    IND_COL_1_CHAUD = Float64[],
    IND_COL_1_FROID = Float64[],
    IND_COL_1_MOYEN = Float64[],
    IND_COL_3 = Float64[],
    IND_ENC = Float64[],
    TYPE_MAINTENANCE = Int[],
    NB_MAINTENANCE_AVANT = Int[]
)

In [ ]:
using Plots

# Fonction pour dessiner un demi-disque orienté à gauche ou à droite
function demi_disque_orienté!(x, y, r, orientation, color)
    θ = LinRange(-π/2, π/2, 100)  # Angle de -π/2 à π/2 pour un demi-disque
    if orientation == :left
        xs = x .- r * cos.(θ)
        ys = y .+ r * sin.(θ)
    elseif orientation == :right
        xs = x .+ r * cos.(θ)
        ys = y .+ r * sin.(θ)
    end
    # Ajouter les points de début et de fin pour fermer le polygone
    xs = [x; xs; x]
    ys = [y; ys; y]
    plot!(xs, ys, seriestype = :shape, fillcolor = color, linecolor = color, legend = false)
end

# Données d'exemple
x = 1:10
y = rand(10)

# Création du graphique
p = plot()
for i in 1:length(x)
    demi_disque_orienté!(x[i], y[i], 0.2, :left, :blue)    # Demi-disque orienté à gauche
    demi_disque_orienté!(x[i], y[i], 0.2, :right, :red)   # Demi-disque orienté à droite
end

# Affichage du graphique
display(p)
